# Hacker News: analizando las publicaciones
En el presente proyecto, se planea comparar dos diferentes tipos de post de la página web [Hacker News](https://news.ycombinator.com/), un sitio web muy popular donde los usuarios redactan publicaciones (o "posts", en adelante se empleará ambos indistintamente) los cuales son votados y comentados por la comunidad. Los dos tipos de publicaciones a analizar son `Ask HN` y `Show HN`.
Los usuarios publican `Ask HN` para preguntar y solicitar información a la comunidad, como por ejemplo "¿Qué cursos en línea recomiendan para aprender *pentesting*?". De igual manera, `Show HN` son post que buscan mostrar proyectos, un producto, o simplemente alguna información a la comunidad.
Se comparará especificamente estos dos tipos de post par determinar las siguientes cuestiones:
* ¿Cuál de las dos publicaciones: `Ask HN` o `Show HN`, reciben más comentarios en promedio?
* ¿Las publicaciones creadas en un momento específico reciben más comentarios en promedio?

El conjunto de datos (en adelante, data set) con el que se trabajará ha sido reducido de 300000 filas a aproximadamente 20000 filas debido a que previamente, se eliminó todas las publicaciones sin comentarios, y luego, se eligió aleatoriamente una muestra de la población restante.

## Introducción
Primero, se lee el data set, luego, se convierte a una lista. Se imprimen las primeras filas para echar un rápido vistazo a la estructura de los datos.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


## Separando el encabezado del conjunto de datos

In [2]:
headers = hn[0]
hn = hn[1:] # Se almacena el data set en otra variable
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


En el header, se observa que el data set contiene información como el título, número de puntos, número de comentarios, autor, y la fecha y hora en la que fueron creadas. Los datos útiles son: el título `title`, la fecha y hora `created_at`, y el número de comentarios `num_comments`.
El siguiente paso es explorar en detalle el número de comentarios por cada tipo de post.
## Extrayendo publicaciones Ask HN y Show HN
Primero, se identifica que los mencionados tipos de posts comienzan con `Ask HN` y `Show HN`, luego, se separan cada tipo en listas diferentes. Esto ayuda a un mejor manejo y análisis en posteriores pasos.

In [3]:
# Identificar los posts que empiezan por Ask HN y Show HN,
# separar los datos en listas diferentes
ask_posts, show_posts, other_posts = [], [], []
for row in hn:
    title = row[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts), len(show_posts), len(other_posts))

1744 1162 17194


## Calculando la cantidad promedio de comentarios para publicaciones Ask HN y Show HN
Con los posts separados, se planea calcular el promedio para cada tipo de publicación respectivamente.

In [4]:
total_ask_comments, total_show_comments = 0, 0

for row in ask_posts:
    total_ask_comments += int(row[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts) # promedio Ask HN
print('Promedio de comentarios para posts Ask HN:', avg_ask_comments)

for row in show_posts:
    total_show_comments += int(row[4])
    
avg_show_comments = total_show_comments / len(show_posts) # promedio Show HN
print('Promedio de comentarios para posts Show HN:', avg_show_comments)

Promedio de comentarios para posts Ask HN: 14.038417431192661
Promedio de comentarios para posts Show HN: 10.31669535283993


Por lo que se observa, los post donde se solicita o se pregunta (Ask HN) reciben en promedio 14 comentarios que los que brindan información (Show HN) con 10 comentarios. Debido a que el tipo Ask HN recibe más comentarios, se enfocará la atención y el análisis a estos en los siguientes procedimientos.
## Encontrando la cantidad de comentarios por hora creados para el tipo Ask HN (más comentado)
Ahora, responderemos a la idea de si es posible maximizar la cantidad de comentarios si el post se publica en una hora determinada. Antes que nada, se encuentra la cantidad de publicaciones Ask HN por hora, acompañado del número de comentarios recibidos. Luego, se calcula el promedio de la cantidad de comentarios por hora del día.

In [5]:
import datetime as dt
result_list = []

for row in ask_posts:
    result_list.append([row[6], int(row[4])]) # fecha y número de comentarios
    
counts_by_hour, comments_by_hour = {}, {}
# counts_by hour: número de posts por hora
# comments_by_hour: número de comentarios por hora
time_format = '%m/%d/%Y %H:%M'

for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, time_format).strftime('%H')
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment
    else:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment
        
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

El diccionario anterior indica la cantidad de comentarios por hora para los posts tipo Ask HN.
## Calculando la cantidad promedio de comentarios por hora para las publicaciones Ask HN
A continuación, se crea una lista cuyo contenido será el promedio de comentarios por hora.

In [6]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

## Ordenando e imprimiendo en pantalla los valores

In [7]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]]) # se intercambia de lugar mutuamente

sorted_swap = sorted(swap_avg_by_hour, reverse=True) # ordenado de mayor a menor

sorted_swap

[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [8]:
# Ordenado de valores e impresión de las 5 horas con
# mayor promedio de comentarios
print("Top de las 5 horas con mayor recurrencia de comentarios para 'Ask HN'")
for avg, hr in sorted_swap[:5]:
    print("{}: {:.2f} promedio de comentarios por publicación".format(dt.datetime.strptime(hr, "%H").strftime("%H:%M"), 
                                                                      avg)
         )

Top de las 5 horas con mayor recurrencia de comentarios para 'Ask HN'
15:00: 38.59 promedio de comentarios por publicación
02:00: 23.81 promedio de comentarios por publicación
20:00: 21.52 promedio de comentarios por publicación
16:00: 16.80 promedio de comentarios por publicación
21:00: 16.01 promedio de comentarios por publicación


La hora que recibe la mayor cantidad de comentarios por publicación es las 15:00 (quince horas) con un promedio de 38.59 comentarios por post. Además, existe alrededor de un 60% de incremento del número de comentarios promedio respecto al segundo lugar (02:00).
Según la [documentación](https://www.kaggle.com/hacker-news/hacker-news-posts/home) del data set, la zona horaria empleada es *Eastern Time* en EUA. Por lo que también se puede reescribir 15:00 como 3:00 pm EST.
## Conclusión
En el presente proyecto, se analizó dos tipos de posts: Ask HN y Show HN, para determinar cuál tipo de post y a qué hora recibe en promedio la mayor cantidad de comentarios. Basado en el análisis previo, para maximizar esto se recomienda publicar un post de tipo Ask HN entre las 15:00 y 16:00 (3:00 pm EST y 4:00 pm EST respectivamente).
No obstante, se debe advertir que el conjunto de datos analizado únicamente incluía posts con comentarios.

Después de lo mencionado, es más correcto concluir que, **de las publicaciones que recibieron comentarios, las publicaciones de tipo Ask HN en un rango horario de 15:00 y 16:00 (3:00 pm EST y 4:00 pm EST) recibieron en promedio más comentarios que otro tipo de publicación y hora.**